In [10]:
import requests
import gmaps
import json

API_KEY = '' # Enter your Google Maps API key here

gmaps.configure(api_key = API_KEY) 

doodle_locations = ['7383 NY-96, Victor, NY 14564',
                '705 Calkins Rd, Rochester, NY 14623',
                '1825 S Clinton Ave, Brighton, NY 14620',
                '2160 Fairport Nine Mile Point Rd, Fairport, NY 14450',
                '979 Jackson Rd, Webster, NY 14580']

target_locations = ['1050 Ridge Rd, Webster, NY 14580',
                '2161 Fairport Nine Mile Point Rd, Fairport, NY 14450',
                '7500 Commons Blvd, Victor, NY 14564',
                '3181 Chili Ave, Rochester, NY 14624',
                '2325 Marketplace Dr, Rochester, NY 14623']

wegmans_locations = ['4287 Genesee Valley Plaza Rd, Geneseo, NY 14454',
                '6600 Pittsford Palmyra Rd, Fairport, NY 14450',
                '345 Eastern Blvd, Canandaigua, NY 14424',
                '745 Calkins Rd, Rochester, NY 14623',
                '3175 Chili Ave, Rochester, NY 14624',
                '2157 Penfield Rd, Penfield, NY 14526',
                '900 Holt Rd, Webster, NY 14580']

def _get_geocode(addresses):
    """ Return a series of geocoded objects based on address input """
    geo = []
    for item in addresses:
        geo.append(requests.get('https://maps.googleapis.com/maps/api/geocode/json?address=' + item + '&key=' + API_KEY).json())
    return geo

def _get_lat_long(geo):
    """ Return specific latitude / longitude information for a series of geocoded objects """
    locs = []
    for item in geo:
        data = json.loads(json.dumps(item))
        for response in data['results']:
            location = response['geometry']['location']
            coords = (location['lat'], location['lng'])
            locs.append(coords)
    return locs

def _plot_polys(locations, stroke, fill):
    """ Return a series of gmaps circle objects based on provided lat/long """
    polys = []   
    for loc in locations:
        polys.append(gmaps.Circle(center=loc, radius=22000, stroke_color=stroke, fill_color=fill))
    return polys

def _plot_markers(locations, stroke, fill):
    """ Return a series of gmaps marker objects based on provided lat/long """
    marker_layer = gmaps.symbol_layer(locations,stroke_color=stroke, fill_color=fill)
    return marker_layer

""" Provide datato gmaps, assemble layers"""
doodle_geo = _get_geocode(doodle_locations)
target_geo = _get_geocode(target_locations)
wegmans_geo = _get_geocode(wegmans_locations)

doodle_latlong = _get_lat_long(doodle_geo)
target_latlong = _get_lat_long(target_geo)
wegmans_latlong = _get_lat_long(wegmans_geo)

doodle_radii = gmaps.drawing_layer(_plot_polys(doodle_latlong,'blue','blue'))
target_radii = gmaps.drawing_layer(_plot_polys(target_latlong,'red','red'))
wegmans_radii = gmaps.drawing_layer(_plot_polys(wegmans_latlong,'green','green'))

doodle_markers  = _plot_markers(doodle_latlong,'blue','blue')
target_markers = _plot_markers(target_latlong,'red','red')
wegmans_markers = _plot_markers(wegmans_latlong,'green','green')

""" Render google map object """
fig = gmaps.figure()
fig.add_layer(doodle_markers)
fig.add_layer(target_markers)
fig.add_layer(wegmans_markers)
fig.add_layer(doodle_radii)
fig.add_layer(target_radii)
fig.add_layer(wegmans_radii)
fig

Figure(layout=FigureLayout(height='420px'))